In [1]:
!nvidia-smi

Sun Mar 23 16:03:57 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   59C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

#Imports

In [4]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

#Define

In [5]:
batch_size = 64
num_classes = 10
learning_rate = 0.001
num_epochs = 20

# Device will determine whether to run the training on GPU or CPU.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#Load Dataset/to-do to fix to custom dataset

In [6]:
# Use transforms.compose method to reformat images for modeling,
# and save to variable all_transforms for later use
all_transforms = transforms.Compose([transforms.Resize((32,32)),
                                     transforms.ToTensor(),
                                     transforms.Normalize(mean=[0.4914, 0.4822, 0.4465],
                                                          std=[0.2023, 0.1994, 0.2010])
                                     ])
# Create Training dataset
train_dataset = torchvision.datasets.CIFAR10(root = './data',
                                             train = True,
                                             transform = all_transforms,
                                             download = True)

# Create Testing dataset
test_dataset = torchvision.datasets.CIFAR10(root = './data',
                                            train = False,
                                            transform = all_transforms,
                                            download=True)

# Instantiate loader objects to facilitate processing
train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                           batch_size = batch_size,
                                           shuffle = True)


test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                           batch_size = batch_size,
                                           shuffle = True)

100%|██████████| 170M/170M [00:04<00:00, 41.3MB/s]


#CNN class inheriting from nn

In [17]:
# Creating a CNN class
class ConvNeuralNet(nn.Module):
#  Determine what layers and their order in CNN object
    def __init__(self, num_classes):
        super(ConvNeuralNet, self).__init__()
        self.conv_layer1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3)
        self.conv_layer2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3)
        self.max_pool1 = nn.MaxPool2d(kernel_size = 2, stride = 2)

        self.conv_layer3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
        self.conv_layer4 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3)
        self.max_pool2 = nn.MaxPool2d(kernel_size = 2, stride = 2)

        self.fc1 = nn.Linear(1600, 128)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(128, num_classes)

    # Progresses data across layers
    def forward(self, x):
        out = self.conv_layer1(x)
        out = self.conv_layer2(out)
        out = self.max_pool1(out)

        out = self.conv_layer3(out)
        out = self.conv_layer4(out)
        out = self.max_pool2(out)

        out = out.reshape(out.size(0), -1)

        out = self.fc1(out)
        out = self.relu1(out)
        out = self.fc2(out)
        return out

#Instantiate CNN class

In [8]:
model = ConvNeuralNet(num_classes)

# Set Loss function with criterion
criterion = nn.CrossEntropyLoss()

# Set optimizer with optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay = 0.005, momentum = 0.9)

total_step = len(train_loader)

#Training

In [10]:
for epoch in range(num_epochs):
# Load in the data in batches using the train_loader object
    for i, (images, labels) in enumerate(train_loader):
        # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device)

        # Move the model to the configured device
        model = model.to(device) # This line is added to move the model to the GPU

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

Epoch [1/20], Loss: 1.8930
Epoch [2/20], Loss: 1.4949
Epoch [3/20], Loss: 1.5860
Epoch [4/20], Loss: 1.4868
Epoch [5/20], Loss: 1.3496
Epoch [6/20], Loss: 1.1079
Epoch [7/20], Loss: 1.0235
Epoch [8/20], Loss: 1.0097
Epoch [9/20], Loss: 1.7313
Epoch [10/20], Loss: 0.4997
Epoch [11/20], Loss: 0.9091
Epoch [12/20], Loss: 1.2004
Epoch [13/20], Loss: 0.5632
Epoch [14/20], Loss: 0.8383
Epoch [15/20], Loss: 0.5343
Epoch [16/20], Loss: 0.8353
Epoch [17/20], Loss: 0.8141
Epoch [18/20], Loss: 0.6089
Epoch [19/20], Loss: 0.7425
Epoch [20/20], Loss: 0.5483


#Testing

In [11]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the {} train images: {} %'.format(50000, 100 * correct / total))

Accuracy of the network on the 50000 train images: 82.792 %


#Using ONNX runtime for production

In [14]:
!pip install onnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 76.5 MB/s eta 0:00:00


#Exporting file for production use

In [15]:

dummy_input = torch.randn(1, 3, 32, 32)  # Example: 1 image, 3 channels, 32x32 size

# Move the dummy input to the same device as the model
dummy_input = dummy_input.to(device) # Move dummy_input to device

# Export the model to ONNX formatm
torch.onnx.export(model, dummy_input, "cifar10_model.onnx", verbose=True)

print("Model saved as cifar10_model.onnx")

Model saved as cifar10_model.onnx


# Only for google drive

In [16]:
# prompt: download that file to my system

from google.colab import files
files.download('cifar10_model.onnx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>